<a href="https://colab.research.google.com/github/Dr-Carlos-Villasenor/VisionComputacional/blob/main/L08_Transferencia_de_aprendizaje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visión Computacional
## Dr. Carlos Villaseñor
### Lección 8 - Transferencia de aprendizaje

Importar bibliotecas



In [5]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import mobilenet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

Importar MobileNet base pero descartaremos las últimas capa

In [ ]:
base_model=mobilenet.MobileNet(weights='imagenet',include_top=False)
print(base_model.summary())

Crear nueva Red Neuronal

In [9]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x) 
x=Dense(512,activation='relu')(x) 
preds=Dense(3, activation='softmax')(x)

Pegar Redes Neuronales

In [ ]:
model=Model(inputs=base_model.input, outputs=preds)
print(model.summary())

Definir que capas se entrenan

In [11]:
for layer in model.layers[:10]:
    layer.trainable=False
for layer in model.layers[10:]:
    layer.trainable=True

Cargar datos nuevos

- Carge el archivo 'train.zip' en la pestaña archivos en el lateral izquierdo
- Corra la siguiente linea para descomprimir los archivos

In [ ]:
!unzip train.zip 

Crear generador desde carpeta

In [13]:
train_datagen=ImageDataGenerator(zoom_range=0.2, horizontal_flip=True,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 preprocessing_function=preprocess_input)

train_generator=train_datagen.flow_from_directory('./train/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)



Found 197 images belonging to 3 classes.


Compilar Modelo y entrenar desde carpeta

In [14]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',
              metrics=['accuracy'])

step_size_train=train_generator.n//train_generator.batch_size

model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=6)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/6
6/6 [==============================] - 36s 416ms/step - loss: 1.0771 - accuracy: 0.6364
Epoch 2/6
6/6 [==============================] - 3s 403ms/step - loss: 0.2501 - accuracy: 0.9333
Epoch 3/6
6/6 [==============================] - 3s 399ms/step - loss: 0.3694 - accuracy: 0.8909
Epoch 4/6
6/6 [==============================] - 3s 458ms/step - loss: 0.7555 - accuracy: 0.8021
Epoch 5/6
6/6 [==============================] - 3s 400ms/step - loss: 0.2855 - accuracy: 0.8848
Epoch 6/6
6/6 [==============================] - 3s 404ms/step - loss: 0.1695 - accuracy: 0.9212
